# Creating a basic chat experience with context variables

In this example, we show how you can build a simple chat bot by sending and updating context with your requests. 

We introduce the Context Variables object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The context is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the context on disk so that you can bring it into your applications.  

In this chat scenario, as the user talks back and forth with the bot, the context gets populated with the history of the conversation. During each new run of the kernel, the context can provide the AI with its variables' content. 

In [ ]:
#!python -m pip install semantic-kernel==0.4.5.dev0

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
deployment_name = os.environ["AZURE_OPENAI_DEPLOYMENT"]
key = os.environ["OPENAI_API_KEY"]

In [2]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    OpenAIChatCompletion,
)

kernel = sk.Kernel()

useAzureOpenAI = True

# Configure AI service used by the kernel
if useAzureOpenAI:
    kernel.add_chat_service(  # We are adding a text service
    "Azure_curie",  # The alias we can use in prompt templates' config.json
    AzureChatCompletion(
        deployment_name=deployment_name,  # Azure OpenAI *Deployment name*
        endpoint=endpoint,  # Azure OpenAI *Endpoint*
        api_key=key,  # Azure OpenAI *Key*
        ),
    )
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service(
        "gpt-3.5",
        OpenAIChatCompletion(ai_model_id="gpt-3.5-turbo", api_key=api_key, org_id=org_id),
    )

Let's define a prompt outlining a dialogue chat bot.

In [3]:
sk_prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

Register your semantic function

In [4]:
chat_function = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    function_name="ChatBot",
    max_tokens=2000,
    temperature=0.7,
    top_p=0.5,
)

Initialize your context

In [5]:
context = kernel.create_new_context()
context["history"] = ""

Chat with the Bot

In [6]:
context["user_input"] = "Hello, my name is Sebastián, and I'm looking for your advice"
bot_answer = await chat_function.invoke_async(context=context)
print(bot_answer)

Variable `$history` not found


Hello Sebastián! I'm here to help. What advice are you looking for?


Update the history with the output

In [7]:
context["history"] += f"\nUser: {context['user_input']}\nChatBot: {bot_answer}\n"
print(context["history"])


User: Hello, my name is Sebastián, and I'm looking for your advice
ChatBot: Hello Sebastián! I'm here to help. What advice are you looking for?



Keep Chatting!

In [8]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    context["user_input"] = input_text

    # Process the user message and get an answer
    answer = await chat_function.invoke_async(context=context)

    # Show the response
    print(f"ChatBot: {answer}")

    # Append the new interaction to the chat history
    context["history"] += f"\nUser: {input_text}\nChatBot: {answer}\n"

In [9]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: Sure! One interesting aspect of ancient Greece is their system of government. Have you heard of the concept of democracy?


In [10]:
await chat("Yes! sure. How about finance: can I ask some simple question about finance?")

User: Yes! sure. How about finance: can I ask some simple question about finance?
ChatBot: Of course! Feel free to ask any finance-related questions you have.


In [11]:
await chat("What are the main topic of Public Filling 10 K?")

User: What are the main topic of Public Filling 10 K?
ChatBot: The main topics covered in a Public Filling 10-K include financial statements, management's discussion and analysis of financial condition and results of operations, risk factors, legal proceedings, and other important information about the company's business.


In [12]:
await chat("What type of financial statements?")

User: What type of financial statements?
ChatBot: The financial statements included in a Public Filling 10-K typically include the balance sheet, income statement, cash flow statement, and statement of stockholders' equity. These statements provide a comprehensive overview of a company's financial performance and position.


After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [13]:
print(context["history"])


User: Hello, my name is Sebastián, and I'm looking for your advice
ChatBot: Hello Sebastián! I'm here to help. What advice are you looking for?

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: Sure! One interesting aspect of ancient Greece is their system of government. Have you heard of the concept of democracy?

User: Yes! sure. How about finance: can I ask some simple question about finance?
ChatBot: Of course! Feel free to ask any finance-related questions you have.

User: What are the main topic of Public Filling 10 K?
ChatBot: The main topics covered in a Public Filling 10-K include financial statements, management's discussion and analysis of financial condition and results of operations, risk factors, legal proceedings, and other important information about the company's business.

User: What type of financial statements?
ChatBot: The financial statements included in a Public Filling 10-K typically include the balance